# Angle-based Joint and Individual Variation Explained

JIVE is a useful algorithm that decomposes multiple views of data into three main categories:
    - Joint Variation
    - Individual Variation
    - Noise
This notebook will prove out the implementation of JIVE and show some examples of the algorithm's usefulness

In [6]:
%load_ext autoreload
%autoreload 2
import mvlearn.jive

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
